In [1]:
import os 
from pprint import pprint

Check that output from other examples has been generated 

In [2]:
from pathlib2 import Path

In [3]:
need_files = ['ethane_struc.json']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run buildingblocks_example.ipynb")
        os.system("jupyter nbconvert --to python  buildingblocks_example.ipynb")
        os.system("python buildingblocks_example.py")

In [4]:
need_files = ['local_res.json','remote_res.json']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run resource_example.ipynb")
        os.system("jupyter nbconvert --to python  resource_example.ipynb")
        os.system("python resource_example.py")

In [5]:
import streamm

Now let's create project and resource to keep track of our work

In [6]:
nwchem_example = streamm.Project('nwchem_example')
res_local = streamm.Resource('local')

In [7]:
res_local.make_dir()

Update relative location of templates directory 

In [8]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

Make sure this is the location of the templates directory that comes with the streamm git repository  https://github.com/NREL/streamm-tools

In [9]:
print res_local.dir['templates']

/Users/tkemper/Development/streamm-tools/examples/../templates/


Create the local directories that will store our files 

In [10]:
nwchem_example.make_dir()

Tell the project about our directories 

In [11]:
nwchem_example.set_resource(res_local)

Read in the ethane structure created in the  buildingblocks_example.ipynb example 

In [12]:
ethane = streamm.Buildingblock('ethane')

In [13]:
ethane.import_json()

In [14]:
print ethane.print_properties()

 n_particles:8 
 n_bonds:7
 n_angles:12
 n_dihedrals:9
 n_impropers:0


Set the paramkeys so we can identify force field parameters later on

In [15]:
for pkey,p in ethane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [16]:
for pk,p in ethane.particles.iteritems():
    p.residue = 1
    p.resname = 'ETH'

Set `rsite`'s to hydrogens to be replaced during join

In [17]:
ethane.particles[1].rsite = 'RH'
ethane.particles[5].rsite = 'RH'

Run `find_rsites()` to populate `func` list

In [18]:
ethane.find_rsites()

In [19]:
print ethane.show_rsites()

rsite:RH[ paticle:atom H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom H (H) index:5 n_bonds:1] 



In [20]:
import copy

Create octane from ethane 

Copy ethane to a new Buildingblock octane 

In [21]:
octane = copy.deepcopy(ethane)

In [22]:
from streamm.structures.buildingblock import attach

Then attach 3 more ethanes to make an octane 

In [23]:
for i in range(3):
    octane = attach(octane,ethane,'RH',1,'RH',0)

In [24]:
octane.tag = 'octane'

In [25]:
print octane.n_particles

26


In [26]:
octane.write_xyz()

Update the tag

In [27]:
octane.tag = 'octane'

Rename the residue and resname for octane

In [28]:
for pk,p in octane.particles.iteritems():
    p.residue = 2
    p.resname = "OCT"

In [29]:
octane.write_xyz()

Create NWChem Calculation object 

In [30]:
nwchem_octane = streamm.NWChem('nw_octane_OPT')

Add calculation to project

In [31]:
nwchem_example.add_calc(nwchem_octane)

Set the structure of the calculation to octane

In [32]:
nwchem_octane.strucC = octane

Set the resource to be local

In [33]:
nwchem_octane.set_resource(res_local)

Make the local directories 

In [34]:
nwchem_octane.make_dir()

Change to the `scratch` directory

In [35]:
os.chdir(nwchem_octane.dir['scratch'])

Copy the template files to the scratch direcotry 

In [36]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_octane.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [37]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_octane.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Read in the template files and add them to the `str` dictionary

In [38]:
nwchem_octane.load_str('templates','nw')        
nwchem_octane.load_str('templates','run')

Set the properties dictionary to desired calculation details

In [39]:
nwchem_octane.properties['basis'] = '6-31g'
nwchem_octane.properties['method'] = 'UHF'
nwchem_octane.properties['charge'] = 0
nwchem_octane.properties['spin_mult'] = 1
nwchem_octane.properties['task'] = 'SCF optimize'
nwchem_octane.properties['coord'] = nwchem_octane.strucC.write_coord()

In [40]:
pprint(nwchem_octane.properties)

{u'allocation': u'',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C       4.02000001       1.81801533      -3.32560073 \n     H       5.22000000       1.81801533      -3.32560073 \n     H       3.62000002       1.22964308      -4.29194336 \n     H       3.62000002       2.94907872      -3.35197474 \n     C       3.57333334       1.21201022      -2.21706715 \n     H       3.97333333       1.80038247      -1.25072452 \n     H       3.97333333       0.08094683      -2.19069315 \n     C       2.23333334       1.21201022      -2.21706715 \n     H       1.83333335       0.62363797      -3.18340979 \n     H       1.83333335       2.34307361      -2.24344116 \n     C       1.78666667       0.60600511      -1.10853358 \n     H       2.18666666       1.19437736      -0.14219095 \n     H       2.18666666      -0.52505828      -1.08215957 \n     C       0.44666667       0.60600511      -1.10853358 \n     H       

Replace the keys in the template strings and write the input files 

In [41]:
nwchem_octane.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_octane.tag))

Add the input file to the properties to be written into the run file

In [42]:
nwchem_octane.properties['input_nw'] = nwchem_octane.files['input']['nw']
nwchem_octane.replacewrite_prop('run','scripts','run','%s.sh'%(nwchem_octane.tag))

Add the log file to the files dictionary 

In [43]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_octane.tag)
nwchem_octane.add_file(file_type,file_key,file_name)

Change back to the root directory and write a json file

In [44]:
os.chdir(nwchem_example.dir['home'])
nwchem_example.export_json()

{u'calculations': {'nw_octane_OPT': u'nwchem'},
 u'meta': {'date': '2017-11-14T19:36:56.244639',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['local']}

Change back to scratch

In [45]:
print nwchem_example.dir['scratch']

/Users/tkemper/Development/streamm-tools/examples


In [46]:
os.chdir(nwchem_example.dir['scratch'])

Run the bash script for the calculation or submit the job to the cluster

In [47]:
print nwchem_octane.tag

nw_octane_OPT


In [48]:
nwchem_octane.run()

Check the status of all the calculations in the project

In [49]:
nwchem_example.check()

Calculation nw_octane_OPT has status written


Run the analysis

In [50]:
nwchem_octane.analysis()

File nw_octane_OPT.log not found 


Tar and zip the results and copy them to a storage location

In [51]:
nwchem_example.store()

Save json in home directory

In [52]:
os.chdir(nwchem_example.dir['home'])
nwchem_example.export_json()

{u'calculations': {'nw_octane_OPT': u'nwchem'},
 u'meta': {'date': '2017-11-14T19:36:56.244639',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['local']}